В этом задании будет использоваться датасет digits из sklearn.datasets. 

- Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы **реализованного вами** 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

In [2]:
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np


In [70]:
def write_answer(file_name, *answ):
    with open(file_name, "w") as file:
        file.write(" ".join(list(map(str, answ))))

In [4]:
print(load_digits()["DESCR"])

.. _digits_dataset:

Optical recognition of handwritten digits dataset
--------------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 5620
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
http://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each bloc

In [6]:
X, y = load_digits().data, load_digits().target

In [63]:
X.shape

(1797, 64)

In [64]:
i = int( X.shape[0] * 0.75 ) + 1
X_train, X_test = np.split(X, [i])
y_train, y_test = np.split(y, [i])

In [65]:
X_train.shape, y_train.shape

((1348, 64), (1348,))

### Задание 1

**Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.**

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. 

- За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [22]:
def euqlid(x: np.ndarray, y: np.ndarray):
    return np.sqrt(
        np.sum(
            np.power(x - y,
                     2)
        )
    )

In [37]:
euqlid(X_train[0], X_train[0],)

0.0

In [67]:
%%time

y_pred = []
for test_object in X_test:
    i_ans = -1
    min_euqlid = float("inf")
    
    for i, train_object in enumerate(X_train):
        cur_euqlid = euqlid(test_object, train_object)
        
        if cur_euqlid < min_euqlid:
            min_euqlid = cur_euqlid
            i_ans = i
            
    y_pred.append(y_train[i_ans])
y_pred = np.array(y_pred)

Wall time: 4.98 s


In [71]:
answer_1 = 1 - accuracy_score(y_test, y_pred)
write_answer("1.txt", answer_1)
print("1nn error:", answer_1)

1nn error: 0.03786191536748329


**Верно. Обратите внимание - ошибка составляет менее 5%! Это показательный пример, что никогда не стоит забывать о бейзлайнах - иногда легко податься соблазну запустить несколько сложных алгоритмов, получить в них качество 90+%, и верить в то, что задача решена хорошо. При этом же запросто может оказаться, что простые методы дают еще лучший результат.**


## Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [75]:
%%time
y_forest_pred = RandomForestClassifier(n_estimators=1000).fit(X_train, y_train).predict(X_test)

Wall time: 2.15 s


In [74]:
answer_2 = 1 - accuracy_score(y_test, y_forest_pred)
print("Random forest error:", answer_2)
write_answer("2.txt", answer_2)

Random forest error: 0.07126948775055675


**Обратите внимание, что качество заметно хуже, чем у 1NN. Это нестандартная ситуация, как правило Random Forest отрабатывает лучше, но такие ситуации иногда встречаются.**